# Wikipedia Analysis

# Steps to implement **Page Rank Algorithm**
* Check Data Raw structure
* Extract relevant data: [Document ID, List of links]
* Transform *List of links* to *List od Docuemts ID*: *Forward Links Table*
* Calculate *Number of output links*
* Construct *Reverse Links Table* from *Forward Links Table*
* Initialize *Page Rank Table*
* Recalculate *Page Rank Table* until:
 * All the *Page Rank* values are stable
 * Reach number of iterations (sugested value: 20 iterations)

# A) Library configuration

In [ ]:
import pandas as pd
import re
import math 

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.types import ArrayType, StringType,LongType, FloatType
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from operator import add 

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "false")

# 1) Check Data Raw Structure
We will use Databrics Wikipedia dataset, which contains 2012 Wikipedia Database in english

Here we define the wikipediaDF Spark Dataframe, with the full database content:
We need to know the total number of documents in this database.

In [ ]:
wikipediaDF=spark.read.parquet("dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet")

To program the PageRank Algorithm, we need to extract a subset of the full database. We will select 0.00001 of the full database, and to avoid randomness behaviour, we set to a fixed seed value=0.

**Note:** For the final evaluation, should change the fraction to 0.001

In [ ]:
PartialWikipediaDF=wikipediaDF.sample(fraction=0.001,seed=0).cache()

Now, we can check the data structure:

## Conclusions with the data raw analysis:
* There are several columns, but the relevant information is stored in the follow columns:
 * **title**: The title of the document.
 * **id**: Id of the document
 * **text**: The content of the document. The most relevant information here (for the page rank algorithm) are the links to other documents. The link is enclosed in brackets, and contains the title of the document.

# 2) Extract Relevant Data
From the previous conclusions, we knows we need to select just three columns [*title*,*id*,*text*], and the relevant information from the *text* column are the links, identified by the titles enclosed in brackets.

Here we need to use regular expressions to select the relevant information from the *text* column.

Here we will implement a *parse_links* function, who receive a string and return a list of strings with the titles of the pointed documents.

This is a Python function, so is not direct callable from the Spark Dataframe, so, we need to define also the User Defined Function, to be usable in Spark Dataframes.

In [ ]:
def parse_links(document_body):
  document_body_u = document_body.lower() 
  data=re.findall(r'\[\[(?!category|wikipedia|file|help|special)((?:(?![\[\]]).)*)\]\]',document_body_u)
  if (len(data)>0):
    links=[s.split('|')[0].split('#')[0].lower() for s in data]
  else:
    links=[]
  return links

In the above function, we used a regular expression to detect more complex Wikipedia links. This allows us to distinguish between compound links and add them to the list that will be used to calculate the forward links table later. 

In [ ]:
parse_links_udf = udf(parse_links,ArrayType(StringType()))

It is necesarly convert the text to lowercase (both: *title* and *text* columns)

In [ ]:
tolower_udf= udf(lambda x: x.lower())

Now, we create parseDF with the selected information, renaming the result tables to "title", and "links"

In [ ]:
parsedDF = PartialWikipediaDF.select(tolower_udf("title").alias("title"),"id",parse_links_udf("text").alias("links"))

# 3) Transform *List of links* to *List od Docuemts ID*: *Forward Links Table*

To get the *id* of the target documents, we need analyse the full Wikipedia Database and extract a table with this two information.

This information is static, and used in distributed way. So, we will collect the data and convert to a Pandas Dataframe (PDF suffix).

In [ ]:
titleidDF=wikipediaDF.select("id",tolower_udf("title").alias("title"));

In [ ]:
titleidPDF=titleidDF.toPandas()

In [ ]:
broadcast_title_idPDF = sc.broadcast(titleidPDF)

In [ ]:
def text_links_2idx_2(links):
  title_idxPDF = broadcast_title_idPDF.value
  if ( len(links)>0):
  # This command looks in the title column if the elements in the list links exists, and if it exists
  # gets his id value. The result is converted to a list.
    result = title_idxPDF[title_idxPDF.title.isin(links)].id.to_list()
  else:
    result = [] 
  return result

In [ ]:
udf_text_links_2idx =udf(text_links_2idx_2,ArrayType(LongType()))

In [ ]:
ForwardDF = parsedDF.select("id",udf_text_links_2idx("links").alias("links")).cache()

Once verified the function, we need to define the UDF function, to invoke it from the **parsedDF** to select just [id, list of ids].

To be efficient, we can broadcast the variable *titleidPDF*, to call it in the transformation.

We will call this Dataframe as: **ForwardDF**

# 4) Calculate *Number of output links*
Using the **ForwardDF**, we need to calculate the number of output links per document. 
Because we will need this information to calculate the PageRank we will collect this information in a Pandas Dataframe, and define a Broadcast variable.

In [ ]:
ForwardDF_WC= ForwardDF.select("id","links",F.size("links").alias("n_succesors"))

# 5) Construct *Reverse Links Table* from *Forward Links Table*
Now, we will define the Reverse Links Table Dataframe (**ReverseDF**), transforming the **ForwardDF** to a Dataframe with [*id*,*list of ids*] or similar.

*Suggestion*: Maybe the *list of ids* could contains not only the id of the target document, also the number of output links. This will improve the Page Rank calcule.

In [ ]:
def reverseId(id,links):
  if (len(links)>0):
    reverse = [ (tgt_id,id) for tgt_id in links ]
  else:
    reverse=[]
  return reverse

In [ ]:
ForwardRDD = ForwardDF_WC.rdd

In [ ]:
ReverseRDD=(ForwardRDD
 .flatMap(lambda r: reverseId(r.id,r.links))
 .groupByKey()
 .map(lambda r: (r[0],list(r[1])))
 )


In [ ]:
reverseDF=spark.createDataFrame(ReverseRDD,["id","links"])

In [ ]:
reverseDF_WC= reverseDF.select("id","links",F.size("links").alias("n_precessors")).withColumnRenamed("links","precessors").withColumnRenamed("id","id1")

In [ ]:
joineddf = reverseDF_WC.join(ForwardDF_WC, ForwardDF_WC.id == reverseDF_WC.id1, how='full')
YesId1DF = joineddf.filter("id1 is NOT NULL")
NoId1DF = joineddf.filter("id1 is NULL")
NoId1DF_v1 = NoId1DF.withColumn("id1", NoId1DF.id)
FullId1DF= YesId1DF.union(NoId1DF_v1)
ReverseDF = FullId1DF.select("id1", "precessors", "n_precessors", "links", "n_succesors").withColumnRenamed("links","succesors").withColumnRenamed("id1","id")

In [ ]:
ReverseDF.show()

# 6) Initialize *Page Rank Table*
We define a Pandas DataFrame (**PageRankPDF**) with the ids of documents in the **ReverseDF**, with the initial value. 

This could be:

\\(\frac{0.85}{N}\\)

where *N* is the number of documents in the **ReverseDF**.

In [ ]:
ReverseDF= ReverseDF.select("id", "precessors", "n_precessors","n_succesors").withColumn("rank", F.lit(0.2)).na.fill(0).cache()

Now we should remove the repeated values from the predecessors of each list, otherwise, the algorithm can easily be overwhelmed and tricked by having a webpage that mentions itself several times.

In [ ]:
ReversePDF= ReverseDF.toPandas()

In [ ]:
line = ReversePDF[ReversePDF["id"]== 204228]["precessors"].tolist()[0]
line[0]

# 7) Recalculate *Page Rank Table* until:
 * All the *Page Rank* values are stable
 * Reach number of iterations (sugested value: 20 iterations)

Should define a PageRank function, who receives the *id* of the document, *list of links*, and current PageRank table (**PageRankPDF**)*, and returns the new PageRank of the document *id* 
The loop must use the **ReverseDF** as master of information.

Now, we define a loop:
* While the conditions of exit are false:
 * Calculate the new page ranks, invoking the PageRank function from the **ReverseDF**, creating a new Dataframe NewPageRankDF with the [id,new_pagerank] info. (Could contains more information, if you use the suggestion in section 5) )
 * Collect the new page ranks and compares it with the previous **PageRankPDF**, checking if the new Page Ranks vary more than a threshold. If not, the exit condition is complied.
 * Update the **PageRankPDF** with the new values,and update the UDF functions.

In [ ]:
def new_pagerank_wd(id1, pagerank, reversePDF):
    '''
    1 - Iterar por los diferentes ids de los preccessors 
    2 - Hacer el sumatorio ponderado (formula de PageRank) 
    '''
    list_of_ids= reversePDF[reversePDF["id"]== id1]["precessors"].tolist()[0]
    d = 0.85 
    
    suma= 0.0
    new_rank = 0.0
    
    if list_of_ids != None:     
      no_repeated_list = list(set(list_of_ids))
      for k in no_repeated_list:
        a = int(k) 
        if int(id1) != a:
          s= pagerank[pagerank["id"]== a]["n_succesors"].tolist()
          r= pagerank[pagerank["id"]== a]["rank"].tolist()
          if int(s[0]) != 0: 
            suma += float(r[0]/s[0])
            
      new_rank = (1-d)+ d * suma
          
    return float(new_rank)

In [ ]:
def new_pagerank(id1, pagerank, reversePDF):
    '''
    1 - Iterar por los diferentes ids de los preccessors 
    2 - Hacer el sumatorio ponderado (formula de PageRank) 
    '''
    list_of_ids= reversePDF[reversePDF["id"]== id1]["precessors"].tolist()[0]
    
    suma= 0.0
    new_page_rank = 0.0
    if list_of_ids != None:     
      no_repeated_list = list(set(list_of_ids))
      for k in no_repeated_list:
        a = int(k) 
        temp= pagerank
        if int(id1) != a:
          s= pagerank[pagerank["id"]== a]["n_succesors"].tolist()
          r= pagerank[pagerank["id"]== a]["rank"].tolist()
          if int(s[0]) != 0:          
            suma += float(r[0]/s[0]) 
       
    return float(suma)

In [ ]:
def new_pagerank_nosuccessors(pagerank,size):
    '''
    Takes into account all the ids that do not have any succesor, and shares their rank with the whole dataframe  
    '''
    # First we add the corresponding value for the pages with no successors 
    withoutNextDF = pagerank.filter(pagerank["n_succesors"]==0)
    #temp= withoutNextDF.select("rank").rdd.reduce(add)
    #sum= 0.0
    #for i in temp: sum += i 
    value = withoutNextDF.select(F.sum(withoutNextDF["rank"]).alias("rank")).collect()[0]["rank"]
    
    UnionDF = pagerank.withColumn("rank", pagerank.rank + F.lit(value/size))
    
    # Then we will normalize the data set so that the greatest value of the rank will be 1 
    max_val = UnionDF.agg({"rank" : "max"}).first()[0]
    newpagerank= UnionDF.withColumn("rank", UnionDF.rank/max_val)
    
    return newpagerank

In [ ]:
math.sqrt(9)

In [ ]:
count= 0
temp= 2
# broadcast_PageRankPDF= sc.broadcast(ReversePDF)
PageRankPDF= ReversePDF
size = ReversePDF.shape[0]
PrevRankPDF= PageRankPDF
temps1= []
temps2= []
temps3= []

while (count < 20) and (temp > 0.001):
    
    # First we take into account the contribution of the usual pages
    # udf_new_pagerank = udf(lambda l: new_pagerank(l, PageRankPDF), FloatType())
    udf_new_pagerank = udf(lambda l: new_pagerank_wd(l, PageRankPDF, ReversePDF), FloatType())
    # NewPageRankDF = ReverseDF.select("id",udf_new_pagerank("precessors").alias("rank"),"n_succesors").cache()
    NewPageRankDF = ReverseDF.select("id", udf_new_pagerank("id").alias("rank"),"n_succesors").cache()
    
    # Then the contribution of the non successor pages 
    #NewPageRankDF_2 = new_pagerank_nosuccessors(NewPageRankDF,size)
    #PageRankPDF = NewPageRankDF_2.toPandas() 
    PageRankPDF = NewPageRankDF.toPandas()
    
    # temp = abs(PageRankPDF["rank"] - PrevRankPDF["rank"]).sum()
    temp1 = abs((abs(PageRankPDF["rank"].sum()) - abs(PrevRankPDF["rank"].sum()))/size) *1.0 
    temps1.append([temp1]) 
    temp2 = abs((abs(PageRankPDF["rank"].sum()) - abs(PrevRankPDF["rank"].sum()))/math.sqrt(size)) *1.0 
    temps2.append([temp2]) 
    temp3 = abs(((abs(PageRankPDF["rank"])- abs(PrevRankPDF["rank"]))/abs(PageRankPDF["rank"])).sum())
    temps3.append([temp3])
    
    PrevRankPDF = PageRankPDF
    count += 1 
        
# FinalPageRankDF = NewPageRankDF_2.orderBy(F.desc("rank")).select("id", "rank")
FinalPageRankDF = NewPageRankDF.orderBy(F.desc("rank")).select("id", "rank")

In [ ]:
FinalPageRankDF.show()
print("count")
print(count)
print("temp")
print(temp)
print("size")
print(size)


In [ ]:
ReverseDF.filter("id == 19279158").show()

## Conclusion

PageRank is one of the underlying algorithms used to rank pages on the internet and it consists of determining the relevance of a website with respect to other websites. This technique uses a directly proportional evaluation system between relevance and evaluation which implies that the higher a website is on the PageRank, the more relevant it is to the end-user. In this notebook, we proposed a possible rudimentary implementation of the PageRank algorithm. 

We first extract the data from the Wikipedia raw file and we extract the links that are on the page, making sure we don't obtain links to the same page in order to remove uni-page infinite cycles. Once we obtain the links in the page, we continue to the next page and extract its respective links and so on. This yields the Forward Links Table. From this table, we can calculate which links take me to the original site for each link in the table and effectively, construct a Reverse Links Table. Once we have both of these tables, we can then execute the PageRank algorithm iteratively. The algorithm changes the value of the rank using the formula above, and it stops when the data has reached a stable value. We attempted this by calculating a "delta" between iterations until the difference between the data was negligible. The end result of the algortihm returns a table with the pages order by rank. Using the small subset, we can confirm that the #1 ranked page also has one of the highest numbers of pages. 